# Block [1-6]
We start by defining our crossover strategy in backtesting library and then importing the algorithmic labeled and model predicted dataframes to apply the backtest and compare the results

In [1]:
# Importing neccessary libraries for backtesting and set logs to only show error messages
from backtesting import Backtest, Strategy
import numpy as np
import pandas as pd
from backtesting.test import SMA
import datetime
import math
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

/usr/local/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
#Here is where we define required parameters for our backtest strategy, This will also help visuazlizing them on the chart
def BULL(array):
    return pd.Series(array)
def BEAR(array):
    return pd.Series(array)

In [3]:
# The strategy we use for opening positions based on our bull or bear labels
class bullbearStrategy(Strategy):
    def init(self):
        self.sma9 = self.I(SMA, self.data.sma9, 1)
        self.sma20 = self.I(SMA, self.data.sma20, 1)
        self.bull = self.I(BULL, self.data.bull)
        self.bear = self.I(BEAR, self.data.bear)
        
    def next(self):
        if self.position.is_long == False:
            if self.data.bull[-1] == 1  :
                self.buy(size=1, limit=None, stop=None,  sl=None , tp=None)
        else:
            if self.data.bear[-1] == 1:
                self.position.close()
                self.bullCounter = 0
                self.bullGap = 0
        if self.position.is_short == False:   
            if self.data.bear[-1] == 1  :
                self.sell(size=1, limit=None, stop=None,  sl=None , tp=None)
        else:
            if self.data.bull[-1] == 1:
                self.position.close()
                self.bullCounter = 0
                self.bullGap = 0

In [4]:
#read predicted data
predicted_source = pd.read_csv('predicted.csv')
predicted_df = predicted_source.copy()
#set date and index
predicted_df['Date'] = pd.to_datetime(predicted_source['Date'], format='%Y-%m-%d %H:%M:%S')
predicted_df.set_index('Date', inplace=True)
# Rename OHLCV columns for backtesting visualiztion purpose
predicted_df.rename(columns={'close': 'Close', 'open': 'Open', 'high': 'High', 'low': 'Low', 'volume': 'Volume'},inplace=True)
predicted_df

,Close,Open,High,Low,Volume,sma9,sma20,sma200,sma200_4h,diff_sma9,diff_sma20,diff_sma200,diff_sma200_4h,diff_sma9_sma20,target_long,target_short,probability_long,probability_short,predicted_long,predicted_short
Date,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,28995.13,28923.63,29031.34,28690.17,2311.811442,28870.590000,28853.1485,26215.70440,21657.24225,0.004314,0.004921,0.106021,0.338819,0.000604,1,0,0.957096,0.000060,1,0
2021-01-01 01:00:00,29409.99,28995.13,29470.00,28960.35,5403.068447,28984.966667,28875.0380,26244.80740,21657.24225,0.014664,0.018526,0.120602,0.357975,0.003807,1,0,0.972548,0.000060,1,0
2021-01-01 02:00:00,29194.65,29410.00,29465.26,29120.03,2384.231552,29054.153333,28891.8200,26272.75530,21657.24225,0.004836,0.010482,0.111214,0.348032,0.005619,1,0,0.984557,0.000121,1,0
2021-01-01 03:00:00,29278.40,29195.25,29367.00,29150.02,1461.345068,29099.281111,28897.9775,26301.36275,21657.24225,0.006155,0.013164,0.113189,0.351899,0.006966,1,0,0.984561,0.000121,1,0
2021-01-01 04:00:00,29220.31,29278.41,29395.00,29029.40,2038.046776,29135.112222,28912.6775,26329.95435,21714.10715,0.002924,0.010640,0.109774,0.345683,0.007693,1,0,0.984551,0.000121,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31 20:00:00,36675.06,36865.75,36980.76,36557.30,1697.834604,36818.311111,36047.6645,36853.66570,47450.29380,-0.003891,0.017405,-0.004846,-0.227085,0.021379,1,0,0.890098,0.000339,1,0
2021-05-31 21:00:00,36932.55,36677.85,37087.30,36623.01,1483.171068,36835.594444,36154.0725,36870.11810,47450.29380,0.002632,0.021532,0.001693,-0.221658,0.018850,1,0,0.812024,0.000338,1,0
2021-05-31 22:00:00,36879.23,36932.55,37250.00,36835.85,2322.342409,36853.144444,36262.0340,36885.05470,47450.29380,0.000708,0.017020,-0.000158,-0.222782,0.016301,1,0,0.811832,0.000338,1,0


In [5]:
#Read the raw labeled dataset which is only labeled based on crossover strategy
labeled_source = pd.read_csv('labeled.csv')
labeled_df = labeled_source.copy()
#set date and index
labeled_df['Date'] = pd.to_datetime(labeled_source['Date'], format='%Y-%m-%d %H:%M:%S')
labeled_df.set_index('Date', inplace=True)
labeled_df

,Open,High,Low,Close,Volume,Numtr,sma9,sma20,sma50,sma200,true_range,atr14,rsi,candle_14,target_long,target_short
Date,,,,,,,,,,,,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,171.0,NaN,NaN,NaN,NaN,52.30,NaN,NaN,NaN,0,0
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,102.0,NaN,NaN,NaN,NaN,37.32,NaN,NaN,NaN,0,0
2017-08-17 06:00:00,4315.32,4345.45,4309.37,4324.35,7.229691,36.0,NaN,NaN,NaN,NaN,36.08,NaN,NaN,NaN,0,0
2017-08-17 07:00:00,4324.35,4349.99,4287.41,4349.99,4.443249,25.0,NaN,NaN,NaN,NaN,62.58,NaN,NaN,NaN,0,0
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,28.0,NaN,NaN,NaN,NaN,44.53,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-10 19:00:00,67790.44,68013.64,66542.85,66650.99,2960.305540,127841.0,67695.636667,67129.1930,67230.7274,63612.51460,1470.79,719.008571,43.797253,-47.144097,1,0
2021-11-10 20:00:00,66650.98,67047.27,64885.98,65840.26,8166.899500,245290.0,67617.132222,67071.3650,67229.2726,63626.72430,2161.29,845.315714,37.616805,-101.685233,1,0
2021-11-10 21:00:00,65840.26,66200.01,62822.90,64251.23,11892.234860,374311.0,67378.715556,66933.0085,67197.5970,63631.01275,3377.11,1054.336429,28.983693,-207.924145,1,0


In [6]:
# Read the label corrected dataframe which includes labels that was corrected manually 
corrected_source = pd.read_csv('labeled_corrected.csv')
corrected_df = corrected_source.copy()
#set date and index
corrected_df['Date'] = pd.to_datetime(corrected_source['Date'], format='%Y-%m-%d %H:%M:%S')
corrected_df.set_index('Date', inplace=True)
corrected_df

,Open,High,Low,Close,Volume,Numtr,sma9,sma20,sma50,sma200,true_range,atr14,rsi,candle_14,target_long,target_short,long_labeled,short_labeled
Date,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,3701.23,3713.00,3689.88,3700.31,686.367415,5534.0,3703.870000,3730.2370,3747.9822,3788.18005,23.12,39.537857,41.701622,-2.285828,0,0,NaN,NaN
2019-01-01 01:00:00,3700.20,3702.73,3684.22,3689.69,613.539120,5086.0,3698.705556,3727.3490,3747.8696,3786.94660,18.51,35.044286,39.464378,-2.873269,0,0,NaN,NaN
2019-01-01 02:00:00,3689.67,3695.95,3675.04,3690.00,895.302181,6391.0,3693.560000,3723.9765,3747.7436,3785.63015,20.91,34.936429,39.566298,-2.644464,0,0,NaN,NaN
2019-01-01 03:00:00,3690.00,3699.77,3685.78,3693.13,796.714817,5709.0,3688.183333,3720.9105,3747.4602,3784.44710,13.99,34.937857,40.652770,-2.232002,0,0,NaN,NaN
2019-01-01 04:00:00,3692.32,3720.00,3685.94,3692.71,1317.452909,7908.0,3688.765556,3717.9355,3746.9590,3783.21580,34.06,33.369286,40.547430,-2.044716,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30 20:00:00,43532.32,43772.47,43100.00,43435.08,1425.997980,55774.0,43334.913333,43288.4545,42410.9756,42886.52925,672.47,525.530000,57.205175,37.676411,1,0,43435.08,NaN
2021-09-30 21:00:00,43436.56,43799.93,43375.10,43754.00,920.783630,29560.0,43372.076667,43327.8840,42454.0158,42893.43780,424.83,539.244286,61.013857,57.659524,1,0,43754.00,NaN
2021-09-30 22:00:00,43754.00,43915.89,43631.14,43741.70,1044.987890,39705.0,43445.402222,43344.5775,42493.4348,42900.56120,284.75,512.302143,60.789149,52.662415,1,0,43741.70,NaN


# Block [7-11]
Prepare the data to feed to backtest strategy defined before and run backtests on each sets of dataframes

In [7]:
# Define the bull and bear columns for each dataframe to make it easier to do backtest on all ot them with only one rule defined
predicted_df['bull'] = predicted_df['predicted_long']
predicted_df['bear'] = predicted_df['predicted_short']

labeled_df['bull'] = labeled_df['target_long']
labeled_df['bear'] = labeled_df['target_short']

corrected_df['bull'] = corrected_df['target_long']
corrected_df['bear'] = corrected_df['target_short']

In [8]:
# chunk a section of each dataframe to run the backtest on it
date_time_from = datetime.datetime.strptime('2021-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
date_time_to = datetime.datetime.strptime('2021-04-01 00:00:00', '%Y-%m-%d %H:%M:%S')
#The sections for running backtests on all our dataframes must be on the same dates in order to have a comparison in win rate and other parameters
predicted_sect = predicted_df.loc[date_time_from:date_time_to,:].copy()
labeled_sect = labeled_df.loc[date_time_from:date_time_to,:].copy()
corrected_sect = corrected_df.loc[date_time_from:date_time_to,:].copy()

In [9]:
#Run the backtest for predicted data
bt = Backtest(predicted_sect, bullbearStrategy,cash=200000, commission=0, exclusive_orders=True)
stats =bt.run()

print(stats)

Start                     2021-01-01 00:00:00
End                       2021-04-01 00:00:00
Duration                     90 days 00:00:00
Exposure Time [%]                    99.90745
Equity Final [$]                    214031.76
Equity Peak [$]                     240843.24
Return [%]                            7.01588
Buy & Hold Return [%]               104.24792
Return (Ann.) [%]                   31.461538
Volatility (Ann.) [%]               25.339123
Sharpe Ratio                         1.241619
Sortino Ratio                        2.563962
Calmar Ratio                         2.730612
Max. Drawdown [%]                  -11.521789
Avg. Drawdown [%]                   -0.893982
Max. Drawdown Duration       18 days 04:00:00
Avg. Drawdown Duration        1 days 15:00:00
# Trades                                   71
Win Rate [%]                        45.070423
Best Trade [%]                      34.617109
Worst Trade [%]                     -6.862995
Avg. Trade [%]                    

In [10]:
#Run the backtest for labeled data
bt = Backtest(labeled_sect, bullbearStrategy,cash=200000, commission=0, exclusive_orders=True)
stats =bt.run()

print(stats)

Start                     2021-01-01 00:00:00
End                       2021-04-01 00:00:00
Duration                     90 days 00:00:00
Exposure Time [%]                    99.90745
Equity Final [$]                    220882.74
Equity Peak [$]                     238042.42
Return [%]                           10.44137
Buy & Hold Return [%]               104.24792
Return (Ann.) [%]                   49.586567
Volatility (Ann.) [%]               26.910276
Sharpe Ratio                         1.842663
Sortino Ratio                        4.505205
Calmar Ratio                          6.80485
Max. Drawdown [%]                   -7.286945
Avg. Drawdown [%]                   -1.018717
Max. Drawdown Duration       16 days 00:00:00
Avg. Drawdown Duration        1 days 20:00:00
# Trades                                  119
Win Rate [%]                        37.815126
Best Trade [%]                      20.148514
Worst Trade [%]                     -7.129611
Avg. Trade [%]                    

In [11]:
#Run the backtest for corrected data
bt = Backtest(corrected_sect, bullbearStrategy,cash=200000, commission=0, exclusive_orders=True)
stats =bt.run()

print(stats)

Start                     2021-01-01 00:00:00
End                       2021-04-01 00:00:00
Duration                     90 days 00:00:00
Exposure Time [%]                    99.90745
Equity Final [$]                    306584.88
Equity Peak [$]                     307532.98
Return [%]                           53.29244
Buy & Hold Return [%]               104.24792
Return (Ann.) [%]                  455.653216
Volatility (Ann.) [%]               69.511783
Sharpe Ratio                          6.55505
Sortino Ratio                      169.032364
Calmar Ratio                       183.469928
Max. Drawdown [%]                   -2.483531
Avg. Drawdown [%]                   -0.461184
Max. Drawdown Duration        7 days 00:00:00
Avg. Drawdown Duration        0 days 15:00:00
# Trades                                   25
Win Rate [%]                            100.0
Best Trade [%]                      38.371122
Worst Trade [%]                      0.082092
Avg. Trade [%]                    